## Internship Spell Checker
Name-Rahul Anand<br>
PhoneNo-9834842796<br>
Email- rahulanand1103@gmail.com<br>

## Getting the data From GoogleDrive

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
download = drive.CreateFile({'id': '1Xk9-HNEum0Hxztd13oHLkzQmHb3C7uWC'})
download.GetContentFile('aliceInWonderland.txt')

In [0]:
download = drive.CreateFile({'id': '1eFILg31DKYZ4aDzVQVNBasRC69ad0saO'})
download.GetContentFile('GoogleNews-vectors-negative300.bin')

In [0]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## 1.Word2vec Approach

In [0]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)

words = model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i

WORDS = w_rank

In [0]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    "Probability of `word`."
    # use inverse of rank as proxy
    # returns 0 if the word isn't in the dictionary
    return - WORDS.get(word, 0)

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [8]:
correction('wonderand') 

'wonderland'

## Misspelled words

### original sentence
<i>In another moment down went Alice after it, never once considering how in the world she was to get out again.</i>

### sentence after mis spelled words

<i>In another <b>momnt</b> <b>doxn went</b> <b>Alie</b> after it, never once <b>considaring</b> how in the world she was to eat to <b>agian</b> </i>


In [0]:
sentence="In another momnt doxn went Alie after it, never once considaring how in the world she was to eat to agian"

In [0]:
import re
listWord = re.sub(r'[.!,;?]',' ',sentence).split()

In [0]:
correctSentence=[]
for word in listWord:
  correctSentence.append(correction(word))
  

In [0]:
correctSentence=' '.join(correctSentence)

In [14]:
correctSentence

'In another moment down went Alie after it never once considering how in the world she was two eat two agian'

## After using word2vec

### original sentence
<i>In another <b>moment</b> <b>down</b> went <b>Alice</b> after it, never once <b>considering</b> how in the world she was to get out <b>again</b>.</i>

### sentence after mis spelled words

<i>In another <b>momnt</b> <b>doxn</b> went <b>Alie</b> after it, never once <b>considaring</b> how in the world she was to get out <b>agian</b> </i>

### sentence after using word2vec
<i>In another <b>moment</b> <b>down</b> went <b>Alie</b> after it never once <b>considering</b> how in the world she was two eat two <b>agian</b></i>


In [0]:
originalSentence='In another moment down went Alice after it, never once considering how in the world she was to get out again.'

misSpellSentence='In another momnt doxn went Alie after it, never once considaring how in the world she was to eat out agian'

correctSentence='In another moment down went Alie after it never once considering how in the world she was to eat out agian'

In [0]:
originalWord = re.sub(r'[.!,;?]',' ',originalSentence).split()

misSpellWord = re.sub(r'[.!,;?]',' ',misSpellSentence).split()

correctWord = re.sub(r'[.!,;?]',' ',correctSentence).split()

In [17]:
for f, b in zip(originalWord, correctWord):
    if(f!=b):
      print(model.similarity(f, b))

0.20982566
0.3504621
0.3612624
